In [1]:
cd /Users/bastus/Desktop/Forschungsprojekt/genimpro

/Users/bastus/Desktop/Forschungsprojekt/genimpro


In [2]:
%run importwrapper.py
%pylab inline

import MySQLdb
db = MySQLdb.connect(host="localhost", 
                     user="genimpro",
                     passwd="genimpropw#2016",
                     db="genimpro")
c = db.cursor() 

/Library/Python/2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [3]:
from essentia.standard import *
from essentia import Pool, array
import numpy as np
import rp_extract as rp
from scipy.signal import argrelmax 

win_s = 512             # fft size
hop_s = win_s/2           # hop size

samplerate = 44100.0
chromaKeys = ["A","A#","B","C","C#","D","D#","E","F","F#","G","G#"]


def HzToMIDI(freq,tuningFrequency=440.0):
    return log(freq/tuningFrequency)/log(2) * 12 + 69

def intensityThreshold(value,t=0.5):
    if value>t:
        return 1
    else:
        return 0

import collections
def intervalStructure(hpcp_binary):
    root = 0
    intervals = [0,0,0,0,0,0,0,0,0,0,0,0]
    intervalIndices = collections.deque(xrange(12))
    
    for rootIdx,rootKey in enumerate(hpcp_binary):
        if rootKey == 0:
            continue
        root = rootIdx
        intervalIndices.rotate(-1 * rootIdx)
        for i,intervalIdx in enumerate(list(intervalIndices)):
            
           intervals[i] = hpcp_binary[intervalIdx]
        break
    
    return intervals,root   

def normalize(hpcp):
    m = max(hpcp)
    for i in range(len(hpcp)):
        hpcp[i] = hpcp[i] / m
    return hpcp
    
    
loudness = essentia.standard.Loudness()
envelope = essentia.standard.Envelope()
centralMoments = essentia.standard.CentralMoments()
shape = essentia.standard.DistributionShape()
dynComplexity = essentia.standard.DynamicComplexity(frameSize=0.01)
dissonance = essentia.standard.Dissonance()
pitchYin = essentia.standard.PitchYinFFT()
flat = essentia.standard.Flatness()
w = essentia.standard.Windowing()
spec = essentia.standard.Spectrum()
SpectralPeaks = essentia.standard.SpectralPeaks(minFrequency=10.0)
TuningFrequencyExtractor  = essentia.standard.TuningFrequencyExtractor()
PitchFilter = essentia.standard.PitchFilter(useAbsolutePitchConfidence=True)
AfterMaxToBeforeMaxEnergyRatio  = essentia.standard.AfterMaxToBeforeMaxEnergyRatio()
SpectralPeaks = essentia.standard.SpectralPeaks(sampleRate=samplerate)
effectiveDuration = essentia.standard.EffectiveDuration(sampleRate=samplerate,thresholdRatio=0.2) #0.01
ZeroCrossingRate = essentia.standard.ZeroCrossingRate()
Centroid = essentia.standard.Centroid()
Inharmonicity = essentia.standard.Inharmonicity()
Flux = essentia.standard.Flux()
Dissonance = essentia.standard.Dissonance()

    
def phenotypeForCell(cell):

    phenotypeDict={}

    loudnessRaw = []
    peakSaliencesRaw = []
    pitchContourRaw = []
    ZeroCrossingRateRaw = []
    CentroidRaw = []
    InharmonicityRaw = []
    SpectralKurtosisRaw = []
    SpectralFluxRaw = []
    SpectralRoughnessRaw = []
    
    audio = EqualLoudness()(cell)
    rh = rp.rp_extract(audio, samplerate, extract_rp=False, extract_ssd=False, extract_rh=True)["rh"]

    for featureframe in FrameGenerator(cell, frameSize = win_s, hopSize = hop_s):
        loudnessRaw.append(loudness(featureframe))
        ZeroCrossingRateRaw.append(ZeroCrossingRate(featureframe))

        spectrum = spec(w(featureframe))
        peaks = SpectralPeaks(spectrum)
        CM = centralMoments(spectrum)
        
        CentroidRaw.append(Centroid(spectrum))
        
        try:
            InharmonicityRaw.append(Inharmonicity(peaks[0],peaks[1]))
        except:
            InharmonicityRaw.append(0)
        
        SpectralRoughnessRaw.append(Dissonance(peaks[0],peaks[1]))
        SpectralKurtosisRaw.append(shape(CM)[2])
        SpectralFluxRaw.append(Flux(spectrum))
        
    #print SpectralFluxRaw
    cellEnvelope=envelope(cell)
    cellCM = centralMoments(loudnessRaw)

    rd = essentia.standard.RhythmDescriptors()(cell)
    
    #dynamics
    phenotypeDict["DynamicGlobalLoudness"] = dynComplexity(cell)[0]
    phenotypeDict["DynamicComplexity"] = dynComplexity(cell)[1]
    phenotypeDict["DynamicSpread"] = shape(cellCM)[0]
    phenotypeDict["DynamicSkewness"] = shape(cellCM)[1]
    phenotypeDict["DynamicKurtosis"] = shape(cellCM)[2]
    
    
    #rhythmics
    maxima = []
    if (type(rh)==type(np.array([0]))):

        maxima = argrelmax(rh,order=5)
        phenotypeDict["RhythmicComplexity"] = len(maxima[0])

        pitchFreq, pitchConfidence = pitchYin(essentia.array(rh))

        phenotypeDict["RhythmicStability"] = pitchConfidence
        phenotypeDict["RhythmicTempo"] = rd[2] #pitchFreq / 1000.0
        phenotypeDict["RhythmicFlatness"] = flat(essentia.array(rh))
        
        peaks = SpectralPeaks(essentia.array(rh))
        phenotypeDict["RhythmicUniformity"] = Inharmonicity(peaks[0][1:-1],peaks[1][1:-1])
        
    else:
        phenotypeDict["RhythmicComplexity"] = 0
        phenotypeDict["RhythmicStability"] = 0
        phenotypeDict["RhythmicTempo"] = 0
        phenotypeDict["RhythmicUniformity"] = 0
        phenotypeDict["RhythmicFlatness"] = 0
    
    
    #melodics
    tf = TuningFrequencyExtractor(audio)
    tuning = np.mean(tf)
    
    run_predominant_melody = essentia.standard.PredominantPitchMelodia(guessUnvoiced=True,
                                               frameSize=2048,
                                               voiceVibrato = True,
                                               minDuration =40,
                                               harmonicWeight = 0.5,
                                               hopSize=128)
    pitch, confidence = run_predominant_melody(audio)
    pitchContourRaw = PitchFilter(pitch, confidence)
    
    midiNotes = []
    floatingParts = []
    notes = []
    spread = 0
    detuning = 0
    
    for note in HzToMIDI(pitchContourRaw):
        if note > 0:     
            floatingPart = note - int(note)
            notes.append(note)
            
            if floatingPart < 0.6:
                midiNote = int(note)
            else:
                midiNote = int(note) + 1
                floatingPart = 1-floatingPart
            
            midiNotes.append(midiNote)
            floatingParts.append(floatingPart)
            
    if (len(notes)>0):
        spread = np.max(notes) - np.min(notes)
        detuning = np.mean(floatingParts)
    try:
        phenotypeDict["MelodicShape"] = AfterMaxToBeforeMaxEnergyRatio(pitchContourRaw)
    except:
        phenotypeDict["MelodicShape"] = 0
        
    #phenotypeDict["midipitches"] = np.unique(midiNotes)   
    phenotypeDict["MelodicComplexity"] = len(np.unique(midiNotes))
    phenotypeDict["MelodicSpread"] = float(spread)
    phenotypeDict["Detuning"] = float(detuning)
    
    #expression
    phenotypeDict["Density"] = effectiveDuration(cell) * samplerate / len(cell)
    phenotypeDict["ZeroCrossingRate"] = np.average(ZeroCrossingRateRaw,weights=loudnessRaw)
    phenotypeDict["SpectralCentroid"] = np.average(CentroidRaw,weights=loudnessRaw) * samplerate/2
    phenotypeDict["SpectralInharmonicity"] = float(np.average(InharmonicityRaw,weights=loudnessRaw))
    phenotypeDict["SpectralKurtosis"] = float(np.average(SpectralKurtosisRaw,weights=loudnessRaw))
    phenotypeDict["SpectralFlux"] = np.average(SpectralFluxRaw,weights=loudnessRaw)
    phenotypeDict["SpectralRoughness"] = float(np.average(SpectralRoughnessRaw,weights=loudnessRaw))

    
    #harmonics
    hpcp = []
    
    HPCP = essentia.standard.HPCP(size = 12,
                               referenceFrequency = tuning,
                               harmonics = 2,
                               bandPreset = True,
                               minFrequency = 40.0,
                               maxFrequency = 5000.0,
                               splitFrequency = 500.0,
                               weightType = 'cosine',
                               nonLinear = True,
                               sampleRate = samplerate)
    
    for featureframe in FrameGenerator(audio, frameSize = 4096 , hopSize = 2048 ):
        if essentia.isSilent(featureframe):
            continue
        peaks = SpectralPeaks(spec(w(featureframe)))
        hpcp.append(HPCP(peaks[0],peaks[1]))
        
    hpcp = np.average(hpcp,axis=0)
    #hpcp = normalize(hpcp)

    if not isinstance(hpcp, np.ndarray):
        hpcp = np.zeros(12) 
        
    mappedHPCP = map(intensityThreshold, hpcp)
    for k,pitch in enumerate(mappedHPCP):
        #phenotypeDict["HPCP " + chromaKeys[k]] = pitch
        phenotypeDict["HPCP " + str(k)] = int(pitch)
        
    
    normedHPCP = np.asarray(mappedHPCP)      
    phenotypeDict["HarmonicSpread"] = int(np.sum(normedHPCP))
                
    intervals,root = intervalStructure(map(intensityThreshold, hpcp))

    phenotypeDict["MelodicRoot"] = root
    
    for j,interval in enumerate(intervals):
        phenotypeDict["Interval " + str(j)] = interval    

    
    return phenotypeDict
        
    

In [4]:
def getKeylist(phenotypes,printKeys=False):
    keylist = phenotypes[0].keys()
    keylist.sort()
    if (printKeys==True):
        for i,keyname in enumerate(keylist):
            print "%i: %s" % (i,keyname)
    return keylist
    

In [5]:
def normalizeByColumn(values, high=1.0, low=0.0):
    mins = np.min(values, axis=0)
    maxs = np.max(values, axis=0)
    rng = maxs - mins
    scaled = high - (((high - low) * (maxs - values)) / rng)
    
    whereAreZeros = numpy.where(values == 0)[0]
    scaled[whereAreZeros] = 0
    return scaled

In [6]:
import simplejson as json

In [7]:
def normHPCP(hpcp):
    return map(intensityThreshold, hpcp)

In [8]:
def transposeDictToArray(phenotypeDicts):
    phenotypeArray = []
    keylist = getKeylist(phenotypeDicts)
    
    for key in keylist:
        sortedVect = []
        for phenotypeDict in phenotypeDicts:
            sortedVect.append(phenotypeDict[key])
        phenotypeArray.append(sortedVect)

    allPhenotypesVect = np.asarray(phenotypeArray)
    return allPhenotypesVect, keylist

In [9]:
def dictToVect(allPhenotypesDicts):

    keylist = allPhenotypesDicts.keys()
    keylist.sort()
    
    sortedVect = []
    for key in keylist:
        sortedVect.append(allPhenotypesDicts[key])
    
    return sortedVect

In [10]:
def normalizePhenotypes(phenotypes,keysToNormalize):
    phenotypesVect,keylist = transposeDictToArray(phenotypes)
    
    for key in keysToNormalize:
        values = phenotypesVect[keylist.index(key)]
        normedValues = normalizeByColumn(values)

        for i,phenotype in enumerate(phenotypes):
            phenotypes[i][key] = normedValues[i]
        
    return phenotypes

In [11]:
keysToNormalize=['DynamicGlobalLoudness','DynamicSpread','DynamicComplexity','SpectralCentroid','SpectralInharmonicity','SpectralRoughness','SpectralFlux','SpectralKurtosis','ZeroCrossingRate']

In [17]:
def addFollowingRests(phenotypes,cellBoundaries):
    for i in range(0,len(cellBoundaries)-1):
        phenotypes[i]["FollowingRest"] = (cellBoundaries[i+1][0]-cellBoundaries[i][1])/samplerate
    phenotypes[len(phenotypes)-1]["FollowingRest"] = -1
    return phenotypes

In [22]:
for recordingID in genimpro.recordings.listRecodings():
    recordingDetails = genimpro.recordings.getRecordingDetails(recordingID)
    print "Recording " + str(recordingID)
    for track in recordingDetails[4]:
        print track
        phenotypes = []
        cellBoundaries = genimpro.recordings.getSoundcellsForTrack(track)
        
        if len(cellBoundaries)==0:
            continue
            
        ids,phenotypes,durations = genimpro.recordings.getPhenotypesForTrack(track)
        #audio = genimpro.recordings.getAudioForTrack(track,normalize=1)
        
        #for cell in cellBoundaries:
        #    phenotypes.append(phenotypeForCell(audio[cell[0]:cell[1]]))

        #phenotypes = normalizePhenotypes(phenotypes,keysToNormalize)
        phenotypes = addFollowingRests(phenotypes,cellBoundaries)


        #pprint(phenotypes[0])
        
        

        genimpro.recordings.updatePhenotypesForTrack(track,cellBoundaries,phenotypes,dbField='phenotypeNormed')
    
    print "  "

Recording 0
['Marius Werner', 'snaredrum', '1/f199ef77-778a-4fa5-9326-f377388687a8.aiff', 1L, 0L, 1L, Decimal('0'), Decimal('8')]
Updated 56 phenotypes for track 1
['Felix Bratfisch', 'snaredrum', '1/25a85c12-3e51-4aec-9e20-e4643d2dc31d.aiff', 2L, 0L, 2L, Decimal('0'), Decimal('8')]
Updated 42 phenotypes for track 2
  
Recording 1
['Marius Werner', 'snaredrum', '1/f90899e0-c4a5-40f6-bd9c-1ec3a7c40a59.aiff', 3L, 1L, 1L, Decimal('0'), Decimal('8')]
Updated 111 phenotypes for track 3
['Felix Bratfisch', 'snaredrum', '1/2e6c915a-85fa-49f2-9379-9d3b11f0d44e.aiff', 4L, 1L, 2L, Decimal('0'), Decimal('8')]
Updated 59 phenotypes for track 4
  
Recording 2
['Marius Werner', 'snaredrum', '1/522fcde8-7d32-451a-9035-4f0fa7972a5f.aiff', 5L, 2L, 1L, Decimal('0'), Decimal('8')]
Updated 175 phenotypes for track 5
['Felix Bratfisch', 'snaredrum', '1/adf42884-8cfd-422b-b780-e50c8893e663.aiff', 6L, 2L, 2L, Decimal('0'), Decimal('8')]
Updated 137 phenotypes for track 6
  
Recording 3
['Marius Werner', 'sna

## Update Phenotypes

In [11]:
playerIDs=[]
allPhenotypesDicts = []
soundcellIDs = []

for recordingID in genimpro.recordings.listRecodings():
    recordingDetails = genimpro.recordings.getRecordingDetails(recordingID)

    for track in recordingDetails[4]:
        try:
            ids,phenotypes,durations,start,end = genimpro.recordings.getPhenotypesForTrack(track)
            if phenotypes == None: continue
            if type(phenotypes) is list:
                #allPhenotypesDicts.extend(phenotypes)
                soundcellIDs.extend(ids)
                    
            #update
            phenotypesVect,keylist = transposeDictToArray(phenotypes)

            for i,phenotype in enumerate(phenotypes):
                playerIDs.append(track[5])
                phenotypeVect = dictToVect(phenotype)
                hpcpValues = np.vstack((phenotypeVect[8:20]))
                normedHPCP = np.asarray(map(normHPCP, hpcpValues))     
                
                phenotypes[i]["HarmonicSpread"] = int(np.sum(normedHPCP))
                
                intervals,root = intervalStructure(map(intensityThreshold, hpcpValues))
                phenotypes[i]["MelodicRoot"] = root

                
                #pprint(phenotype)
                    
            phenotypesVect,keylist = transposeDictToArray(phenotypes)
            keysToNormalize = keylist[0:8] + keylist[20:22] + keylist[34:35] + keylist[36:]

            #normalise
            phenotypes = normalizePhenotypes(phenotypes,keysToNormalize)
                    
            for i,phenotype in enumerate(phenotypes):        
                updateNormedPhenotype(phenotype,ids[i])
            
        except TypeError:
            continue

NameError: name 'updateNormedPhenotype' is not defined

In [164]:
keysToNormalize = keylist[0:8] + keylist[20:22] + keylist[34:35] + keylist[36:]
print keysToNormalize

['Centroid', 'Density', 'Detuning', 'DynamicComplexity', 'DynamicGlobalLoudness', 'DynamicKurtosis', 'DynamicSkewness', 'DynamicSpread', 'HarmonicSpread', 'Inharmonicity', 'MelodicComplexity', 'MelodicShape', 'MelodicSpread', 'RhythmicComplexity', 'RhythmicFlatness', 'RhythmicStability', 'RhythmicTempo', 'RhythmicUniformity', 'SpectralFlux', 'SpectralKurtosis', 'SpectralRoughness', 'ZeroCrossingRate']


In [153]:
keylist = getKeylist(phenotypes,printKeys=True)

0: Centroid
1: Density
2: Detuning
3: DynamicComplexity
4: DynamicGlobalLoudness
5: DynamicKurtosis
6: DynamicSkewness
7: DynamicSpread
8: HPCP 0
9: HPCP 1
10: HPCP 10
11: HPCP 11
12: HPCP 2
13: HPCP 3
14: HPCP 4
15: HPCP 5
16: HPCP 6
17: HPCP 7
18: HPCP 8
19: HPCP 9
20: HarmonicSpread
21: Inharmonicity
22: Interval 0
23: Interval 1
24: Interval 10
25: Interval 11
26: Interval 2
27: Interval 3
28: Interval 4
29: Interval 5
30: Interval 6
31: Interval 7
32: Interval 8
33: Interval 9
34: MelodicComplexity
35: MelodicRoot
36: MelodicShape
37: MelodicSpread
38: RhythmicComplexity
39: RhythmicFlatness
40: RhythmicStability
41: RhythmicTempo
42: RhythmicUniformity
43: SpectralFlux
44: SpectralKurtosis
45: SpectralRoughness
46: ZeroCrossingRate


In [166]:
def updateNormedPhenotype(phenotype,phenotypeID):

    phenotype = json.dumps(phenotype)
    sqlcommand = "UPDATE soundcells SET phenotypeNormed=%s where ID=%i" % (repr(phenotype),phenotypeID)
    c.execute(sqlcommand)
    db.commit()
        
    #print "Updated phenotype %i" % (phenotypeID)